# Data Generation

In [1]:
import random
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [2]:
# Set random seed for reproducibility
np.random.seed(42)

n_records = 500000

ip_ranges = [
    '128.1.0.', '155.55.0.', '157.20.5.', '157.20.20.', '157.20.30.',
    '192.168.1.', '10.0.0.', '172.16.0.'
]
urls = [
    '/ai-assistant', '/demo-request', '/pricing', '/events',
    '/job-prototype', '/solutions', '/contact', '/about',
    '/ai-assistant/chat', '/demo-request/schedule',
    '/events/upcoming', '/job-prototype/submit'
]

interaction_map = {
    '/ai-assistant': 'AI Assistant Request',
    '/ai-assistant/chat': 'AI Chat Initiated',
    '/demo-request': 'Demo Inquiry',
    '/demo-request/schedule': 'Demo Scheduled',
    '/job-prototype': 'Job Prototype Inquiry',
    '/job-prototype/submit': 'Job Prototype Submitted',
    '/events': 'Event Inquiry',
    '/events/upcoming': 'Event Registered',
    '/pricing': 'Pricing Viewed',
    '/solutions': 'Solutions Browsed',
    '/contact': 'Contact Request',
    '/about': 'About Page Viewed'
}

methods = ['GET', 'POST', 'PUT']
status_codes = [200, 201, 301, 302, 304, 400, 403, 404, 500]
salespeople = ['Maano', 'Bob', 'Charlie', 'Diana', 'Eve', 'Frank']
products = ['AI Suite', 'DataPro', 'InsightX', 'VisionBot', 'CloudSync']
countries = ['USA', 'UK', 'Germany', 'India', 'Canada', 'Australia']
job_types = ['Consultation', 'Integration', 'Support', 'Training', 'Demo']

# Generate dates spanning 36 months (3 years) for three full seasonal cycles
start_date = datetime.now() - timedelta(days=1095)  # 1095 days = 3 years
timestamps = [
    (start_date + timedelta(
        days=random.randint(0, 1095),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59),
        seconds=random.randint(0, 59)))
    for _ in range(n_records)
]

# More realistic URL distribution
url_weights = [0.25, 0.15, 0.12, 0.1, 0.08, 0.08, 0.06, 0.05, 0.03, 0.03, 0.03, 0.02]
random_urls = random.choices(urls, weights=url_weights, k=n_records)

data = {
    'timestamp': [ts.strftime('%d/%m/%Y %H:%M:%S') for ts in timestamps],
    'ip_address': [f"{random.choice(ip_ranges)}{random.randint(1, 255)}" for _ in range(n_records)],
    'method': [random.choices(methods, weights=[0.85, 0.12, 0.03])[0] for _ in range(n_records)],
    'url': random_urls,
    'status_code': [random.choices(status_codes, weights=[0.82, 0.05, 0.03, 0.02, 0.02, 0.02, 0.015, 0.015, 0.01])[0] for _ in range(n_records)],
    'salesperson': [random.choices(salespeople, weights=[0.3, 0.2, 0.2, 0.15, 0.1, 0.05])[0] for _ in range(n_records)],
    'product_sold': [random.choices(products, weights=[0.35, 0.25, 0.2, 0.15, 0.05])[0] for _ in range(n_records)],
    'date_of_sale': [ts.strftime('%d/%m/%Y %H:%M:%S') for ts in timestamps],
    'cost': [round(random.choices([
        random.uniform(500, 1000),
        random.uniform(1000, 2000),
        random.uniform(2000, 3500),
        random.uniform(3500, 5000)
    ], weights=[0.4, 0.3, 0.2, 0.1])[0], 2) for _ in range(n_records)],
    'customer_country': [random.choices(countries, weights=[0.35, 0.25, 0.15, 0.12, 0.08, 0.05])[0] for _ in range(n_records)],
    'job_type_requested': [random.choices(job_types, weights=[0.3, 0.25, 0.2, 0.15, 0.1])[0] for _ in range(n_records)],
    'customer_interaction': [interaction_map[url] for url in random_urls],
}


In [3]:
# Create DataFrame
df = pd.DataFrame(data)

top1000 = df.head(10000)

In [ ]:
import os

def validate_iis_log_csv(file_path, required_columns=None, nrows=100):
    """
    Validates if the CSV at file_path matches expected IIS log structure and is not empty.
    Args:
        file_path (str): Path to the CSV file.
        required_columns (list): List of required column names.
        nrows (int): Number of rows to check for completeness.
    Returns:
        dict: Validation results.
    """
    result = {'file_exists': False, 'structure_valid': False, 'not_empty': False, 'missing_columns': []}
    if not os.path.isfile(file_path):
        return result

    result['file_exists'] = True
    try:
        sample = pd.read_csv(file_path, nrows=nrows)
        result['not_empty'] = not sample.empty
        if required_columns:
            missing = [col for col in required_columns if col not in sample.columns]
            result['missing_columns'] = missing
            result['structure_valid'] = len(missing) == 0
        else:
            result['structure_valid'] = True
    except Exception as e:
        result['error'] = str(e)
    return result

iis_required_columns = [
    'timestamp', 'ip_address', 'method', 'url', 'status_code',
    'salesperson', 'product_sold', 'date_of_sale', 'cost',
    'customer_country', 'job_type_requested', 'customer_interaction'
]
validation = validate_iis_log_csv('ai_solutions_web_sales_logs.csv', required_columns=iis_required_columns)
print(validation)

{'file_exists': True, 'structure_valid': True, 'not_empty': True, 'missing_columns': []}


In [5]:
# Save to CSV
df.to_csv('ai_solutions_web_sales_logs.csv', index=False)

In [6]:
df = pd.read_csv('ai_solutions_web_sales_logs.csv', parse_dates=['date_of_sale'], dayfirst=True)
print(df.shape)
df['month'] = df['date_of_sale'].dt.to_period('M')

(500000, 12)


In [7]:
monthly_sales = df.groupby('month')['cost'].sum().reset_index()

In [8]:
len(monthly_sales)

37

# Data Normalization

In [9]:
# Create dimension tables and fact table
"""
# Date Dimension
date_dim = df['date_of_sale'].unique()
date_dim = pd.DataFrame(date_dim, columns=['date'])
date_dim['date_id'] = date_dim.index
date_dim['date'] = pd.to_datetime(date_dim['date'])
date_dim['year'] = date_dim['date'].dt.year
date_dim['month'] = date_dim['date'].dt.month
date_dim['day'] = date_dim['date'].dt.day
date_dim['day_of_week'] = date_dim['date'].dt.dayofweek
date_dim['day_name'] = date_dim['date'].dt.day_name()
date_dim['month_name'] = date_dim['date'].dt.month_name()

# Customer Location Dimension
location_dim = pd.DataFrame(countries, columns=['country'])
location_dim['location_id'] = location_dim.index

# Salesperson Dimension 
salesperson_dim = pd.DataFrame(salespeople, columns=['salesperson_name'])
salesperson_dim['salesperson_id'] = salesperson_dim.index

# Product Dimension
product_dim = pd.DataFrame(products, columns=['product_name'])
product_dim['product_id'] = product_dim.index

# Job Type Dimension
job_type_dim = pd.DataFrame(job_types, columns=['job_type'])
job_type_dim['job_type_id'] = job_type_dim.index

# Create fact table
fact_table = df.copy()

# Join with dimension tables to get IDs
fact_table['date_id'] = pd.to_datetime(fact_table['date_of_sale']).map(dict(zip(date_dim['date'], date_dim['date_id'])))
fact_table['location_id'] = fact_table['customer_country'].map(dict(zip(location_dim['country'], location_dim['location_id'])))
fact_table['salesperson_id'] = fact_table['salesperson'].map(dict(zip(salesperson_dim['salesperson_name'], salesperson_dim['salesperson_id'])))
fact_table['product_id'] = fact_table['product_sold'].map(dict(zip(product_dim['product_name'], product_dim['product_id'])))
fact_table['job_type_id'] = fact_table['job_type_requested'].map(dict(zip(job_type_dim['job_type'], job_type_dim['job_type_id'])))

# Select only needed columns for fact table
fact_table = fact_table[['date_id', 'location_id', 'salesperson_id', 'product_id', 'job_type_id', 
                        'cost', 'status_code', 'customer_interaction']]
                        
                        
                        """

"\n# Date Dimension\ndate_dim = df['date_of_sale'].unique()\ndate_dim = pd.DataFrame(date_dim, columns=['date'])\ndate_dim['date_id'] = date_dim.index\ndate_dim['date'] = pd.to_datetime(date_dim['date'])\ndate_dim['year'] = date_dim['date'].dt.year\ndate_dim['month'] = date_dim['date'].dt.month\ndate_dim['day'] = date_dim['date'].dt.day\ndate_dim['day_of_week'] = date_dim['date'].dt.dayofweek\ndate_dim['day_name'] = date_dim['date'].dt.day_name()\ndate_dim['month_name'] = date_dim['date'].dt.month_name()\n\n# Customer Location Dimension\nlocation_dim = pd.DataFrame(countries, columns=['country'])\nlocation_dim['location_id'] = location_dim.index\n\n# Salesperson Dimension \nsalesperson_dim = pd.DataFrame(salespeople, columns=['salesperson_name'])\nsalesperson_dim['salesperson_id'] = salesperson_dim.index\n\n# Product Dimension\nproduct_dim = pd.DataFrame(products, columns=['product_name'])\nproduct_dim['product_id'] = product_dim.index\n\n# Job Type Dimension\njob_type_dim = pd.DataFra